In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import gzip
import json

In [3]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

22/12/12 14:52:50 WARN Utils: Your hostname, Zephyrus resolves to a loopback address: 127.0.1.1; using 172.17.140.69 instead (on interface eth0)
22/12/12 14:52:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 14:52:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark_df = spark.read.json('data/Musical_Instruments_5.json')
spark_df.printSchema()
print(f"There are {spark_df.count()} rows of data")

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color Name:: string (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Configuration:: string (nullable = true)
 |    |-- Edition:: string (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Item Display Length:: string (nullable = true)
 |    |-- Item Package Quantity:: string (nullable = true)
 |    |-- Length:: string (nullable = true)
 |    |-- Model Number:: string (nullable = true)
 |    |-- Number of Items:: string (nullable = true)
 |    |-- Package Quantity:: string (nullable = true)
 |    |-- Package Type:: string (nullable = true)
 |    |-- Platform for Display:

In [5]:
spark_df.count()

231392

### Dropping All Columns Except `overall`  and `reviewTest`

In [5]:
import re

In [6]:
# Drop all columns except reviewText and overall
df = spark_df[['overall', 'reviewText']]
df.printSchema()

# Drop all rows with NA values
df = df.dropna()

# Convert to rdd
rdd = df.rdd

# Convert to lowercase
rdd = rdd.map(lambda x: (x[0], x[1].lower()))

# Strip non-alphanumeric & space chars
rdd = rdd.map(lambda x: (x[0], re.sub(r'[^A-Za-z0-9 ]+', '', x[1])))

# Convert overall rating to 1 or 0
# 4-5 -> Positive
# 1-3 -> Negative
rdd = rdd.map(lambda x: (1, x[1]) if x[0] in [4,5] else (0, x[1]))

# Convert review to array of words
rdd = rdd.map(lambda x: (x[0], x[1].split()))

# Filter out empty reviews
rdd = rdd.filter(lambda x: len(x[1]) > 0)

# Cache as basic_rdd
basic_rdd = rdd
basic_rdd.cache

root
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)



<bound method RDD.cache of PythonRDD[17] at RDD at PythonRDD.scala:53>

## Convert to N-Grams

In [74]:
basic_rdd.count()

231307

In [7]:
def ngram(n: int, x: [str]):
    if n == 1:
        return x
    
    result = []
    
    for i in range(len(x)-n+1):
        item = []
        a = i
        for a in range(i, i+n):
            item.append(x[a])
        result.append(item)
    return result

In [57]:
stopwords = []

with open("data/stopwords.txt") as file:
    global stopwords
    stopwords = file.read().splitlines()

def filter_stopwords(x):
    global stopwords
    result = []
    # print(type(x[1][0]))
    # print(isinstance(x[1][0], str))
    for s in x[1]:
        # print(isinstance(s, str))
        if isinstance(s, str):
            if not s in stopwords:
                result.append(s)
            return (x[0], result)
        if not s[0] in stopwords or not s[-1] in stopwords:
            result.append(s)
    return (x[0], result)

In [74]:
ngram_rdd = basic_rdd.filter(lambda x: x[0] == 0)

print(f"Num neg: {ngram_rdd.count()}")

n = 4
ngram_rdd = ngram_rdd.map(lambda x: (x[0], ngram(n, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)


# ngram_rdd_nosw = ngram_rdd.map(lambda x: (x[0], filter_stopwords(x[1])))
# ngram_counts_rdd = ngram_rdd.map(lambda x: len(x[1]))
# ngram_nosw_count_rdd = ngram_rdd_nosw.map(lambda x: len(x[1]))

# def sum(a, b):
#     return a+b

if n == 1:
    ngrams_rdd = ngram_rdd.flatMap(lambda x: x[1]).map(lambda x: (x, 1))
else:
    ngrams_rdd = ngram_rdd.flatMap(lambda x: x[1]).map(lambda x: (tuple(x), 1))
ngram_counts = ngrams_rdd.reduceByKey(sum)

shared_grams = ngram_counts.filter(lambda x: x[1] > 100)

# # count_total = ngrams_rdd.count()
print(f"total ngrams:  {ngram_counts.count()}")
print(f"shared ngrams: {shared_grams.count()}")

for a in shared_grams.sortByKey().collect():
    print(a)

# ngrams_counts = ngrams_rdd.countByKey()

# distinct_ngrams = ngrams_rdd.toDF().distinct()
# print(f"distinct ngrams: {distinct_ngrams.count()}")

# print(f"No SW Filter:\n\tmin:  {ngram_counts_rdd.min()}\n\tmax:  {ngram_counts_rdd.max()}\n\tmean: {ngram_counts_rdd.mean()}")
# print(f"With SW Filter:\n\tmin:  {ngram_nosw_count_rdd.min()}\n\tmax:  {ngram_nosw_count_rdd.max()}\n\tmean: {ngram_nosw_count_rdd.mean()}")

Num neg: 30768


total ngrams:  1389322
shared ngrams: 12
(('at', 'the', 'same', 'time'), 121)
(('dont', 'waste', 'your', 'money'), 112)
(('get', 'what', 'you', 'pay'), 555)
(('i', 'sent', 'it', 'back'), 115)
(('i', 'would', 'not', 'recommend'), 153)
(('if', 'you', 'are', 'looking'), 123)
(('it', 'does', 'the', 'job'), 105)
(('out', 'of', 'the', 'box'), 413)
(('right', 'out', 'of', 'the'), 193)
(('there', 'is', 'no', 'way'), 108)
(('this', 'is', 'a', 'good'), 139)
(('this', 'is', 'a', 'great'), 122)


In [176]:
one_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(1, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)
two_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(2, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)
three_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(3, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)
four_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(4, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)

# print("Reviews w/ 1-grams: " + str(one_gram_rdd.filter(lambda x: x[0] == 0).count()))
# print("Reviews w/ 2-grams: " + str(two_gram_rdd.filter(lambda x: x[0] == 0).count()))
# print("Reviews w/ 3-grams: " + str(three_gram_rdd.filter(lambda x: x[0] == 0).count()))
# print("Reviews w/ 4-grams: " + str(four_gram_rdd.filter(lambda x: x[0] == 0).count()))

print("Reviews w/ 1-grams: " + str(one_gram_rdd.count()))
print("Reviews w/ 2-grams: " + str(two_gram_rdd.count()))
print("Reviews w/ 3-grams: " + str(three_gram_rdd.count()))
print("Reviews w/ 4-grams: " + str(four_gram_rdd.count()))

# for a in rdd.take(3):
    # print(a)

Reviews w/ 1-grams: 230993


Reviews w/ 2-grams: 222503


Reviews w/ 3-grams: 209881


Reviews w/ 4-grams: 203566


In [94]:
filter_stopwords(["a"])

157


In [66]:
rdd = convert_to_n_grams(2, basic_rdd)

4
14
12
133
4
6
62
25
19
27
4
50
4
14
12
7
3
8838

56
10
14
6
89
33
2
29
28
7
9
8
3173

30
37
70
23
118
8
240
105
21
23
5
20
4
4
18
46
4
8
282
32
34
90
42
98
22
56
71
250
6
7
33
14
312
83

28
70
4
22
16
2
237
29
21
10
96
20
23
94
51
84
61
81
84
2
75
38
250
114
10
7622

2
58
2
137
25
7
30
90
5
18
88
174

29
51
4
3
78
5
9
8
6
41
1
29
3
136
2
7
46
68
25
3
105
21
46
43
182
119
49
42
40
45
67
23
146
32
38
59
14
10
16


None
None
None


In [46]:
df.write.json("data/Musical_Instruments_2.json")

In [48]:
df = spark.read.json("data/Musical_Instruments_2.json")

In [51]:
df.printSchema()
print(df.count())

root
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)

231344
231344
